In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud
from textwrap import wrap

In [3]:
df2 = pd.read_csv('MKB.csv')

In [4]:
df2.head()

,Episode,Cleaned_content
0,20211024,dear countrymen namaskar…namaskar billion time...
1,20210926,dear countrymen namaskar aware important progr...
2,20210829,dear countrymen namaskar us know today birth a...
3,20210725,dear countrymen namaskar amazing pictures take...
4,20210627,dear countrymen namaskar often mann ki baat vo...


In [6]:
cv=CountVectorizer(analyzer='word')
data=cv.fit_transform(df2['Cleaned_content'])
df_dtm = pd.DataFrame(data.toarray(), columns=cv.get_feature_names())
df_dtm.index=df2['Episode']
df_dtm.head(3)

,aaba,aadarsh,aadhar,aaguner,aakashvani,aamra,aanganbari,aanganwadi,aapada,aapadratopi,...,सर,सरद,सरलम,सरसम,सस,सहय,हट,हप,हरत,हस
Episode,,,,,,,,,,,,,,,,,,,,,
20211024,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20210926,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20210829,0,0,0,0,0,0,0,0,0,0,...,2,1,1,1,0,0,0,1,0,2


In [ ]:
def generate_wordcloud(data,title):
  wc = WordCloud(width=400, height=330, max_words=150,colormap="Dark2").generate_from_frequencies(data)
  plt.figure(figsize=(10,8))
  plt.imshow(wc, interpolation='bilinear')
  plt.axis("off")
  plt.title('\n'.join(wrap(title,60)),fontsize=13)
  #plt.show()
  plt.savefig('wordcloud/'+title+'.png')
  
# Transposing document term matrix
df_dtm=df_dtm.transpose()

# Plotting word cloud for each product
for index,product in enumerate(df_dtm.columns):
    print("Ep date: ",product)
    generate_wordcloud(df_dtm[product].sort_values(ascending=False),str(product))